In [1]:
import numpy as np
import matplotlib.pyplot as plt
import subprocess
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
import plumed
import nglview as ng
import MDAnalysis as mda

%matplotlib inline
torch.manual_seed(42)

In [3]:
u = mda.Universe('md_0.gro')
print(len(u.trajectory))
nucleic = u.select_atoms('nucleic')

atom = u.atoms[243]  
residue = atom.residue 
res_interesting = nucleic.select_atoms(f"resid {residue.resid}")
view = ng.show_mdanalysis(res_interesting)
view.clear_representations()
view.add_licorice(res_interesting)
view.add_surface(res_interesting,opacity=0.1)
view.add_trajectory(nucleic)

atompairs=[[243,751],[243,784],[243,787],[243,794],
                [243,800],[246,751],[246,784],[246,787],[246,794],[246,800]]

cmap = plt.colormaps.get_cmap('rainbow')
for a,atompair in enumerate(atompairs):
    atoms_temp1 = u.select_atoms(f"index {atompair[0]}")
    atoms_temp2 = u.select_atoms(f"index {atompair[1]}")

    center1 = atoms_temp1[0].position 
    center2 = atoms_temp2[0].position 

    view.shape.add_sphere(
            center1,
            list(cmap(a/len(atompairs))),
            0.5
        )

    view.shape.add_sphere(
            center2,
            list(cmap(a/len(atompairs))),
            0.5
        )

    view.shape.add_arrow(center1, center2, list(cmap(a/len(atompairs))), 0.5)
        
        
        
view

1


NGLWidget()

In [4]:
view.close()

In [45]:
u = mda.Universe('md_0.gro')
print(len(u.trajectory))
nucleic = u.select_atoms('nucleic')
A8_flipping_base = nucleic.select_atoms(f"resid {8} and name C* N* O* P*")
A8_flipping_base_CN = nucleic.select_atoms(f"resid {8} and (name N1 N3 N6 C6 C2)")
C9_neighbor = nucleic.select_atoms(f"resid {9} and name C2 C4 C5 C6 C8 C1' C2' O4'")
G7_neighbor = nucleic.select_atoms(f"resid {7} and name C2 C4 C5 C6 C8 C1' C2' O4'")
C9_neighbor_base = nucleic.select_atoms(f"resid {9} and name N1 N2 N3 N4 N6 N7 N9 O2 O4 O6")
G7_neighbor_base = nucleic.select_atoms(f"resid {7} and name N1 N2 N3 N4 N6 N7 N9 O2 O4 O6")
G24_neighbor = nucleic.select_atoms(f"resid {24} and name N1 N2 N3 N4 N6 N7 N9 O2 O4 O6")
C25_neighbor = nucleic.select_atoms(f"resid {25} and name N1 N2 N3 N4 N6 N7 N9 O2 O4 O6")

print(A8_flipping_base[0])
print(A8_flipping_base_CN[0])
print(C9_neighbor[0])
print(G7_neighbor[0])
print(C9_neighbor_base[0])
print(G7_neighbor_base[0])
print(G24_neighbor[0])
print(C25_neighbor[0])

1
<Atom 225: P of type P of resname A, resid 8 and segid SYSTEM>
<Atom 242: C6 of type C of resname A, resid 8 and segid SYSTEM>
<Atom 267: O4' of type O of resname C, resid 9 and segid SYSTEM>
<Atom 200: O4' of type O of resname G, resid 7 and segid SYSTEM>
<Atom 270: N1 of type N of resname C, resid 9 and segid SYSTEM>
<Atom 203: N9 of type N of resname G, resid 7 and segid SYSTEM>
<Atom 744: N9 of type N of resname G, resid 24 and segid SYSTEM>
<Atom 778: N1 of type N of resname C, resid 25 and segid SYSTEM>


In [46]:
def get_atom_pairs_within_cutoff(atom_list1, atom_list2, cutoff):
    pairs_within_cutoff = []
    
    for i in range(len(atom_list1)):
        atom_i = str(atom_list1[i].id)
        
        for j in range(len(atom_list2)):
            atom_j = str(atom_list2[j].id)
            
            distance = np.linalg.norm(atom_list1[i].position - atom_list2[j].position)
            
            if distance <= cutoff:
                pairs_within_cutoff.append((atom_i, atom_j, distance))
                
    return np.array(pairs_within_cutoff)

A8G24_cmap=get_atom_pairs_within_cutoff(A8_flipping_base_CN,G24_neighbor,5)
A8C25_cmap=get_atom_pairs_within_cutoff(A8_flipping_base_CN,C25_neighbor,4)
A8G24C25_cmap=np.vstack((A8G24_cmap,A8C25_cmap))
G7C9_cmap=get_atom_pairs_within_cutoff(G7_neighbor,C9_neighbor,8.5)
A8G7_cmap=get_atom_pairs_within_cutoff(A8_flipping_base_CN,C9_neighbor_base,4)
A8C9_cmap=get_atom_pairs_within_cutoff(A8_flipping_base_CN,G7_neighbor_base,4)
A8G7C9_cmap=np.vstack((A8G7_cmap,A8C9_cmap))

print(A8G24_cmap.shape)
print(A8C25_cmap.shape)
print(A8G24C25_cmap.shape)
print(G7C9_cmap.shape)
print(A8G7_cmap.shape)
print(A8C9_cmap.shape)
print(A8G7C9_cmap.shape)

(6, 3)
(4, 3)
(10, 3)
(20, 3)
(12, 3)
(8, 3)
(20, 3)


In [47]:
def visualize_atom_pairs(u, atompairs,c_cmap):
    for a, atompair in enumerate(atompairs):
        atoms_temp1 = u.select_atoms(f"id {atompair[0]}")
        atoms_temp2 = u.select_atoms(f"id {atompair[1]}")

        center1 = atoms_temp1[0].position
        center2 = atoms_temp2[0].position

        view.shape.add_sphere(
                center1,
                list(cmap(c_cmap)),
                0.5, a
            )

        view.shape.add_sphere(
                center2,
                list(cmap(c_cmap)),
                0.5, a
            )

        view.shape.add_arrow(center1, center2, list(cmap(c_cmap)), 0.5, a)


In [48]:
u = mda.Universe(dire_analyse+'md_0.gro')
print(len(u.trajectory))
nucleic = u.select_atoms('nucleic')

atom = u.atoms[243]  
residue = atom.residue 
res_interesting = nucleic.select_atoms(f"resid {residue.resid}")
view = ng.show_mdanalysis(res_interesting)
view.clear_representations()
view.add_licorice(res_interesting)
view.add_surface(res_interesting,opacity=0.1)
view.add_trajectory(nucleic)

cmap = plt.colormaps.get_cmap('rainbow')


visualize_atom_pairs(u, A8G24C25_cmap[:,:2],1/5)
visualize_atom_pairs(u, A8G7C9_cmap[:,:2],2/5)
visualize_atom_pairs(u, G7C9_cmap[:,:2],3/5)
        
view

1


NGLWidget()

No such comm: 042528b7788d4a5cbba83a8513eb7f06
No such comm: 042528b7788d4a5cbba83a8513eb7f06
No such comm: 042528b7788d4a5cbba83a8513eb7f06


In [49]:
view.close()

In [50]:
lines=[]

#------------------------------------- additional CVs to track ---------------------------------------
lines.append("""

d1: DISTANCE ATOMS=243,751 #A-G
d2: DISTANCE ATOMS=243,784 #A-C
d3: DISTANCE ATOMS=243,787 #A-C
d4: DISTANCE ATOMS=243,794 #A-RIB
d5: DISTANCE ATOMS=243,800 #A-O
d6: DISTANCE ATOMS=246,751 #A-G
d7: DISTANCE ATOMS=246,784 #A-C
d8: DISTANCE ATOMS=246,787 #A-C
d9: DISTANCE ATOMS=246,794 #A-RIB
d10: DISTANCE ATOMS=246,800 #A-O

d1n: CUSTOM ARG=d1 FUNC=x/2.11 PERIODIC=NO
d2n: CUSTOM ARG=d2 FUNC=x/2.25 PERIODIC=NO
d3n: CUSTOM ARG=d3 FUNC=x/2.17 PERIODIC=NO
d4n: CUSTOM ARG=d4 FUNC=x/2.53 PERIODIC=NO
d5n: CUSTOM ARG=d5 FUNC=x/2.78 PERIODIC=NO
d6n: CUSTOM ARG=d6 FUNC=x/2.07 PERIODIC=NO
d7n: CUSTOM ARG=d7 FUNC=x/2.22 PERIODIC=NO
d8n: CUSTOM ARG=d8 FUNC=x/2.13 PERIODIC=NO
d9n: CUSTOM ARG=d1 FUNC=x/2.49 PERIODIC=NO
d10n: CUSTOM ARG=d10 FUNC=x/2.71 PERIODIC=NO


#hlda: COMBINE ARG=d1,d2,d3,d4,d5,d6,d7,d8,d9,d10 COEFFICIENTS=-14.815268,-15.319241,-13.384423,-7.26893,-7.971926,-16.557427,-14.808337,-14.668623,-9.334746,-9.811748 PERIODIC=NO
hldaN: COMBINE ARG=d1n,d2n,d3n,d4n,d5n,d6n,d7n,d8n,d9n,d10n COEFFICIENTS=-11.63344,-11.311855,-12.316635,-10.216072,-10.565613,-10.745473,-9.307333,-11.834765,-11.317639,-12.201419 PERIODIC=NO

WO: GROUP ATOMS=1020-45543:4 # water molecules
AWO: COORDINATION GROUPA=243 GROUPB=WO SWITCH={RATIONAL D_0=0.0 R_0=0.35 NN=6 MM=10} NLIST NL_CUTOFF=2.0 NL_STRIDE=20

t1: TORSION ATOMS=225,228,229,232 #A, P-O-C-C
t2: TORSION ATOMS=228,229,232,251 #A, O-C-C-C
t3: TORSION ATOMS=229,232,251,257 #A C-C-C_O
t4: TORSION ATOMS=232,251,257,258 #A C-C-O-P
t5: TORSION ATOMS=251,257,258,261 #A C-O-P-O
t6: TORSION ATOMS=257,258,261,262 #A-C  O-P-O-C
t7: TORSION ATOMS=258,261,262,265 #C P-O-C-C
t8: TORSION ATOMS=261,262,265,282 #C O-C-C-C
t9: TORSION ATOMS=262,265,282,288 #C C-C-C-O
t10: TORSION ATOMS=265,282,288,289 #C C-C-O-P

t-1: TORSION ATOMS=218,224,225,228 #G-A C-O-P-O
t-2: TORSION ATOMS=198,218,224,225 #G-A C-C-O-P
    
""")
             
#----------------------------------------------------------------------------------------------

features=[]

temp=[]
for e,elem in enumerate(A8G24C25_cmap):
    lines.append('dA8G24C25_%s: DISTANCE ATOMS=%s,%s \n'%(e+1,elem[0],elem[1]))
    temp.append('dA8G24C25_%s' %(e+1))

temp = ','.join(temp)              
lines.append('A8G24C25_cmap: COMBINE ARG=%s PERIODIC=NO \n' %temp)
features.append('A8G24C25_cmap')

temp=[]
for e,elem in enumerate(A8G7C9_cmap):
    lines.append('dA8G7C9_%s: DISTANCE ATOMS=%s,%s \n'%(e+1,elem[0],elem[1]))
    temp.append('dA8G7C9_%s' %(e+1))

temp = ','.join(temp)              
lines.append('A8G7C9_cmap: COMBINE ARG=%s PERIODIC=NO \n' %temp)
features.append('A8G7C9_cmap')

temp=[]
for e,elem in enumerate(G7C9_cmap):
    lines.append('dG7C9_%s: DISTANCE ATOMS=%s,%s \n'%(e+1,elem[0],elem[1]))
    temp.append('dG7C9_%s' %(e+1))

temp = ','.join(temp)              
lines.append('G7C9_cmap: COMBINE ARG=%s PERIODIC=NO \n' %temp)
features.append('G7C9_cmap')

CV_print=['A8G24C25_cmap','A8G7C9_cmap','G7C9_cmap','hldaN','AWO','t2','t3','t4','t5','t6']
CV_print= ','.join(CV_print)  
lines.append('PRINT FMT=%s STRIDE=1 FILE=COLVAR_stateA ARG=%s \n' %('%8.4f',CV_print))


f = open("ABMD/plumed.dat", "w")
for elem in lines:
    f.writelines(elem)
f.close()

In [10]:
!scp md_0.gro ./ABMD

In [11]:
!plumed driver --igro ABMD/md_0.gro --plumed ABMD/plumed.dat --kt 2.5

PLUMED: PLUMED is starting
PLUMED: Version: 2.10.0-dev (git: d3361c35a) compiled on Mar 17 2023 at 16:25:48
PLUMED: Please cite these papers when using PLUMED [1][2]
PLUMED: For further information see the PLUMED web page at http://www.plumed.org
PLUMED: Root: /home/rizziv@farma.unige.ch/plumed2-2.9/
PLUMED: For installed feature, see /home/rizziv@farma.unige.ch/plumed2-2.9//src/config/config.txt
PLUMED: Molecular dynamics engine: driver
PLUMED: Precision of reals: 8
PLUMED: Running over 1 node
PLUMED: Number of threads: 1
PLUMED: Cache line size: 512
PLUMED: Number of atoms: 45543
PLUMED: File suffix: 
PLUMED: FILE: ABMD/plumed.dat
PLUMED: Action DISTANCE
PLUMED:   with label d1
PLUMED:   between atoms 243 751
PLUMED:   using periodic boundary conditions
PLUMED: Action DISTANCE
PLUMED:   with label d2
PLUMED:   between atoms 243 784
PLUMED:   using periodic boundary conditions
PLUMED: Action DISTANCE
PLUMED:   with label d3
PLUMED:   between atoms 243 787
PLUMED:   using periodic boun

PLUMED:                                               Cycles        Total      Average      Minimum      Maximum
PLUMED:                                                    1     0.019624     0.019624     0.019624     0.019624
PLUMED: 1 Prepare dependencies                             1     0.000064     0.000064     0.000064     0.000064
PLUMED: 2 Sharing data                                     1     0.000214     0.000214     0.000214     0.000214
PLUMED: 3 Waiting for data                                 1     0.000004     0.000004     0.000004     0.000004
PLUMED: 4 Calculating (forward loop)                       1     0.000664     0.000664     0.000664     0.000664
PLUMED: 5 Applying (backward loop)                         1     0.000194     0.000194     0.000194     0.000194
PLUMED: 6 Update                                           1     0.000027     0.000027     0.000027     0.000027


In [12]:
colvar=plumed.read_as_pandas("COLVAR_stateA")

print(colvar['A8G24C25_cmap'].to_numpy())
print(colvar['A8G7C9_cmap'].to_numpy())
print(colvar['G7C9_cmap'].to_numpy())
print(colvar['hldaN'].to_numpy())
print(colvar['AWO'].to_numpy())

[4.1382]
[7.1411]
[15.568]
[-26.7914]
[7.6808]


+++ Loading the PLUMED kernel runtime +++
+++ PLUMED_KERNEL="/home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumedKernel.so" +++
+++ An error occurred. Message from dlopen(): /home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumedKernel.so: undefined symbol: _ZN5torch3jit6MethodclESt6vectorIN3c106IValueESaIS4_EERKSt13unordered_mapINSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEES4_St4hashISD_ESt8equal_toISD_ESaISt4pairIKSD_S4_EEE +++
+++ This error is expected if you are trying to load a kernel <=2.4
+++ Trying /home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumed.so +++


In [51]:
lines=[]

#------------------------------------- additional CVs to track ---------------------------------------
lines.append("""

d1: DISTANCE ATOMS=243,751 #A-G
d2: DISTANCE ATOMS=243,784 #A-C
d3: DISTANCE ATOMS=243,787 #A-C
d4: DISTANCE ATOMS=243,794 #A-RIB
d5: DISTANCE ATOMS=243,800 #A-O
d6: DISTANCE ATOMS=246,751 #A-G
d7: DISTANCE ATOMS=246,784 #A-C
d8: DISTANCE ATOMS=246,787 #A-C
d9: DISTANCE ATOMS=246,794 #A-RIB
d10: DISTANCE ATOMS=246,800 #A-O

d1n: CUSTOM ARG=d1 FUNC=x/2.11 PERIODIC=NO
d2n: CUSTOM ARG=d2 FUNC=x/2.25 PERIODIC=NO
d3n: CUSTOM ARG=d3 FUNC=x/2.17 PERIODIC=NO
d4n: CUSTOM ARG=d4 FUNC=x/2.53 PERIODIC=NO
d5n: CUSTOM ARG=d5 FUNC=x/2.78 PERIODIC=NO
d6n: CUSTOM ARG=d6 FUNC=x/2.07 PERIODIC=NO
d7n: CUSTOM ARG=d7 FUNC=x/2.22 PERIODIC=NO
d8n: CUSTOM ARG=d8 FUNC=x/2.13 PERIODIC=NO
d9n: CUSTOM ARG=d1 FUNC=x/2.49 PERIODIC=NO
d10n: CUSTOM ARG=d10 FUNC=x/2.71 PERIODIC=NO


#hlda: COMBINE ARG=d1,d2,d3,d4,d5,d6,d7,d8,d9,d10 COEFFICIENTS=-14.815268,-15.319241,-13.384423,-7.26893,-7.971926,-16.557427,-14.808337,-14.668623,-9.334746,-9.811748 PERIODIC=NO
hldaN: COMBINE ARG=d1n,d2n,d3n,d4n,d5n,d6n,d7n,d8n,d9n,d10n COEFFICIENTS=-11.63344,-11.311855,-12.316635,-10.216072,-10.565613,-10.745473,-9.307333,-11.834765,-11.317639,-12.201419 PERIODIC=NO

WO: GROUP ATOMS=1020-45543:4 # water molecules
AWO: COORDINATION GROUPA=243 GROUPB=WO SWITCH={RATIONAL D_0=0.0 R_0=0.35 NN=6 MM=10} NLIST NL_CUTOFF=2.0 NL_STRIDE=20

t1: TORSION ATOMS=225,228,229,232 #A, P-O-C-C
t2: TORSION ATOMS=228,229,232,251 #A, O-C-C-C
t3: TORSION ATOMS=229,232,251,257 #A C-C-C_O
t4: TORSION ATOMS=232,251,257,258 #A C-C-O-P
t5: TORSION ATOMS=251,257,258,261 #A C-O-P-O
t6: TORSION ATOMS=257,258,261,262 #A-C  O-P-O-C
t7: TORSION ATOMS=258,261,262,265 #C P-O-C-C
t8: TORSION ATOMS=261,262,265,282 #C O-C-C-C
t9: TORSION ATOMS=262,265,282,288 #C C-C-C-O
t10: TORSION ATOMS=265,282,288,289 #C C-C-O-P

t-1: TORSION ATOMS=218,224,225,228 #G-A C-O-P-O
t-2: TORSION ATOMS=198,218,224,225 #G-A C-C-O-P
    
""")
             
#----------------------------------------------------------------------------------------------


features=[]

temp=[]
for e,elem in enumerate(A8G24C25_cmap):
    lines.append('dA8G24C25_%s: DISTANCE ATOMS=%s,%s \n'%(e+1,elem[0],elem[1]))
    temp.append('dA8G24C25_%s' %(e+1))

temp = ','.join(temp)              
lines.append('A8G24C25_cmap: COMBINE ARG=%s PERIODIC=NO \n' %temp)
features.append('A8G24C25_cmap')

temp=[]
for e,elem in enumerate(A8G7C9_cmap):
    lines.append('dA8G7C9_%s: DISTANCE ATOMS=%s,%s \n'%(e+1,elem[0],elem[1]))
    temp.append('dA8G7C9_%s' %(e+1))

temp = ','.join(temp)              
lines.append('A8G7C9_cmap: COMBINE ARG=%s PERIODIC=NO \n' %temp)
features.append('A8G7C9_cmap')

temp=[]
for e,elem in enumerate(G7C9_cmap):
    lines.append('dG7C9_%s: DISTANCE ATOMS=%s,%s \n'%(e+1,elem[0],elem[1]))
    temp.append('dG7C9_%s' %(e+1))

temp = ','.join(temp)              
lines.append('G7C9_cmap: COMBINE ARG=%s PERIODIC=NO \n' %temp)
features.append('G7C9_cmap')



lines.append('abmd: ABMD ARG=A8G24C25_cmap,A8G7C9_cmap TO=100,100 KAPPA=0.001,0.001 \n')

lines.append('COMMITTOR ...\n')
lines.append('   ARG=A8G24C25_cmap,A8G7C9_cmap \n' )
lines.append('   STRIDE=10\n')
lines.append('   BASIN_LL1=20,32\n')
lines.append('   BASIN_UL1=50,50\n')
lines.append('...\n')

CV_print=['A8G24C25_cmap','A8G7C9_cmap','G7C9_cmap','hldaN','AWO','t2','t3','t4','t5','t6']
CV_print= ','.join(CV_print)  
lines.append('PRINT FMT=%s STRIDE=10 FILE=COLVAR_flip_out1 ARG=%s \n' %('%8.4f',CV_print))


f = open("ABMD/plumed.dat", "w")
for elem in lines:
    f.writelines(elem)
f.close()

In [15]:
!scp md_0.tpr ./ABMD
!scp RNA_solv_ionsfsc.top ./ABMD
!scp index.ndx ./ABMD
!scp md_prod.mdp ./ABMD
!scp md_0.gro ./ABMD
!scp ./ABMD/RNA_solv_ionsfsc.top ./ABMD/topol.top 

In [52]:
%%bash 
gmx_mpi grompp -f ./ABMD/md_prod.mdp -c ./ABMD/md_0.gro -p ./ABMD/topol.top -n ./ABMD/index.ndx -o ./ABMD/flip_out1.tpr 

             :-) GROMACS - gmx grompp, 2022.5-plumed_2.10.0_dev (-:

Executable:   /home/rizziv@farma.unige.ch/programs/gromacs-2022.5/install_single_cuda/bin/gmx_mpi
Data prefix:  /home/rizziv@farma.unige.ch/programs/gromacs-2022.5/install_single_cuda
Working dir:  /home/frohlkin@farma.unige.ch/Projects/TransitionPathSampling_DeepLNE/Publication_Toymodels/RNA
Command line:
  gmx_mpi grompp -f ./ABMD/md_prod.mdp -c ./ABMD/md_0.gro -p ./ABMD/topol.top -n ./ABMD/index.ndx -o ./ABMD/flip_out1.tpr

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'

NOTE 1 [file ./ABMD/md_prod.mdp]:
  leapfrog does not yet support Nose-Hoover chains, nhchainlength reset to 1

Generating 1-4 interactions: fudge = 0.5
Number of degrees of freedom in T-Coupling group System is 69506.00

There was 1 note

Back Off! I just backed up ./ABMD/flip_out1.tpr to ./ABMD/#flip_out1.tpr.1#

GROMACS reminds you: "Marie, you're looking more radiant every day!" (Pierre Curie)



Setting the LD random seed to 2000306175

Generated 666 of the 666 non-bonded parameter combinations

Generated 666 of the 666 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'system1'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'Na+'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'WAT'

turning H bonds into constraints...

Cleaning up constraints and constant bonded interactions with virtual sites

The largest distance between excluded atoms is 0.410 nm

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.037 nm, buffer size 0.037 nm

Set rlist, assuming 4x4 atom pair-list, to 1.000 nm, buffer size 0.000 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 52x52x52, spacing 0.150 0.150 0.150

Estimate for the relative computational 

In [53]:
%%bash 
cat > ABMD/run.sh << EOF
export OMP_NUM_THREADS=8
mpirun -n 2 gmx_mpi mdrun -deffnm flip_out1 -plumed plumed.dat -pin on -pinoffset 0 -notunepme -nsteps 1000000 -nb gpu &
EOF

In [54]:
!find . -type f -name "#*" -exec rm -rf {} +
!find . -type f -name "step*" -exec rm -rf {} +
!find . -type f -name "bck*" -exec rm -rf {} +

In [61]:
colvar=plumed.read_as_pandas("ABMD/COLVAR_flip_out1")
colvar

+++ Loading the PLUMED kernel runtime +++
+++ PLUMED_KERNEL="/home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumedKernel.so" +++
+++ An error occurred. Message from dlopen(): /home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumedKernel.so: undefined symbol: _ZN5torch3jit6MethodclESt6vectorIN3c106IValueESaIS4_EERKSt13unordered_mapINSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEES4_St4hashISD_ESt8equal_toISD_ESaISt4pairIKSD_S4_EEE +++
+++ This error is expected if you are trying to load a kernel <=2.4
+++ Trying /home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumed.so +++


time  A8G24C25_cmap  A8G7C9_cmap  G7C9_cmap     hldaN      AWO      t2  \
0      0.00         4.1382       7.1411    15.5680  -26.7914   7.6808  1.1280   
1      0.02         4.1958       7.1892    15.5495  -26.8412   7.5576  1.1946   
2      0.04         4.2118       7.2670    15.5246  -26.6719   7.5793  1.2844   
3      0.06         4.2089       7.2567    15.6583  -26.4885   7.5350  1.2491   
4      0.08         4.2260       7.2770    15.7327  -26.5962   7.4721  1.3349   
...     ...            ...          ...        ...       ...      ...     ...   
3284  65.68        19.8960      32.8707    14.1020 -102.1523  14.6843  1.0132   
3285  65.70        19.9019      32.9071    13.9694 -102.0909  14.6257  0.9998   
3286  65.72        19.9343      33.0082    13.7123 -102.1064  14.5782  1.0213   
3287  65.74        19.9872      33.0489    13.5859 -102.4853  14.7055  0.8691   
3288  65.76        20.0776      33.1314    13.5824 -102.9212  14.7798  0.9766   

          t3      t4      t5      t6  
0     2.5027 -3.0143 -1.8910 -1.1955  
1     2.7003 -2.8910 -2.1728 -1.1477  
2     2.7184 -2.7966 -2.2823 -1.2526  
3     2.6639 -2.6906 -2.2632 -1.3178  
4     2.7477 -2.8081 -2.3336 -1.3031  
...      ...     ...     ...     ...  
3284  1.8832 -2.7734  1.4621 -1.3103  
3285  1.9604 -2.8075  1.4432 -1.3879  
3286  1.8906 -2.8223  1.3518 -1.4281  
3287  1.9919 -2.8350  1.3628 -1.3794  
3288  1.9019 -2.7979  1.4195 -1.3504  

[3289 rows x 11 columns]

In [57]:
lines=[]

#------------------------------------- additional CVs to track ---------------------------------------
lines.append("""

d1: DISTANCE ATOMS=243,751 #A-G
d2: DISTANCE ATOMS=243,784 #A-C
d3: DISTANCE ATOMS=243,787 #A-C
d4: DISTANCE ATOMS=243,794 #A-RIB
d5: DISTANCE ATOMS=243,800 #A-O
d6: DISTANCE ATOMS=246,751 #A-G
d7: DISTANCE ATOMS=246,784 #A-C
d8: DISTANCE ATOMS=246,787 #A-C
d9: DISTANCE ATOMS=246,794 #A-RIB
d10: DISTANCE ATOMS=246,800 #A-O

d1n: CUSTOM ARG=d1 FUNC=x/2.11 PERIODIC=NO
d2n: CUSTOM ARG=d2 FUNC=x/2.25 PERIODIC=NO
d3n: CUSTOM ARG=d3 FUNC=x/2.17 PERIODIC=NO
d4n: CUSTOM ARG=d4 FUNC=x/2.53 PERIODIC=NO
d5n: CUSTOM ARG=d5 FUNC=x/2.78 PERIODIC=NO
d6n: CUSTOM ARG=d6 FUNC=x/2.07 PERIODIC=NO
d7n: CUSTOM ARG=d7 FUNC=x/2.22 PERIODIC=NO
d8n: CUSTOM ARG=d8 FUNC=x/2.13 PERIODIC=NO
d9n: CUSTOM ARG=d1 FUNC=x/2.49 PERIODIC=NO
d10n: CUSTOM ARG=d10 FUNC=x/2.71 PERIODIC=NO


#hlda: COMBINE ARG=d1,d2,d3,d4,d5,d6,d7,d8,d9,d10 COEFFICIENTS=-14.815268,-15.319241,-13.384423,-7.26893,-7.971926,-16.557427,-14.808337,-14.668623,-9.334746,-9.811748 PERIODIC=NO
hldaN: COMBINE ARG=d1n,d2n,d3n,d4n,d5n,d6n,d7n,d8n,d9n,d10n COEFFICIENTS=-11.63344,-11.311855,-12.316635,-10.216072,-10.565613,-10.745473,-9.307333,-11.834765,-11.317639,-12.201419 PERIODIC=NO

WO: GROUP ATOMS=1020-45543:4 # water molecules
AWO: COORDINATION GROUPA=243 GROUPB=WO SWITCH={RATIONAL D_0=0.0 R_0=0.35 NN=6 MM=10} NLIST NL_CUTOFF=2.0 NL_STRIDE=20

t1: TORSION ATOMS=225,228,229,232 #A, P-O-C-C
t2: TORSION ATOMS=228,229,232,251 #A, O-C-C-C
t3: TORSION ATOMS=229,232,251,257 #A C-C-C_O
t4: TORSION ATOMS=232,251,257,258 #A C-C-O-P
t5: TORSION ATOMS=251,257,258,261 #A C-O-P-O
t6: TORSION ATOMS=257,258,261,262 #A-C  O-P-O-C
t7: TORSION ATOMS=258,261,262,265 #C P-O-C-C
t8: TORSION ATOMS=261,262,265,282 #C O-C-C-C
t9: TORSION ATOMS=262,265,282,288 #C C-C-C-O
t10: TORSION ATOMS=265,282,288,289 #C C-C-O-P

t-1: TORSION ATOMS=218,224,225,228 #G-A C-O-P-O
t-2: TORSION ATOMS=198,218,224,225 #G-A C-C-O-P
    
""")
             
#----------------------------------------------------------------------------------------------


features=[]

temp=[]
for e,elem in enumerate(A8G24C25_cmap):
    lines.append('dA8G24C25_%s: DISTANCE ATOMS=%s,%s \n'%(e+1,elem[0],elem[1]))
    temp.append('dA8G24C25_%s' %(e+1))

temp = ','.join(temp)              
lines.append('A8G24C25_cmap: COMBINE ARG=%s PERIODIC=NO \n' %temp)
features.append('A8G24C25_cmap')

temp=[]
for e,elem in enumerate(A8G7C9_cmap):
    lines.append('dA8G7C9_%s: DISTANCE ATOMS=%s,%s \n'%(e+1,elem[0],elem[1]))
    temp.append('dA8G7C9_%s' %(e+1))

temp = ','.join(temp)              
lines.append('A8G7C9_cmap: COMBINE ARG=%s PERIODIC=NO \n' %temp)
features.append('A8G7C9_cmap')

temp=[]
for e,elem in enumerate(G7C9_cmap):
    lines.append('dG7C9_%s: DISTANCE ATOMS=%s,%s \n'%(e+1,elem[0],elem[1]))
    temp.append('dG7C9_%s' %(e+1))

temp = ','.join(temp)              
lines.append('G7C9_cmap: COMBINE ARG=%s PERIODIC=NO \n' %temp)
features.append('G7C9_cmap')


lines.append('abmd: ABMD ARG=A8G24C25_cmap,A8G7C9_cmap,G7C9_cmap TO=100,100,0 KAPPA=0.001,0.001,10 \n')

lines.append('COMMITTOR ...\n')
lines.append('   ARG=A8G24C25_cmap,A8G7C9_cmap,G7C9_cmap \n' )
lines.append('   STRIDE=10\n')
lines.append('   BASIN_LL1=20,32,5\n')
lines.append('   BASIN_UL1=50,50,8\n')
lines.append('...\n')

CV_print=['A8G24C25_cmap','A8G7C9_cmap','G7C9_cmap','hldaN','AWO','t2','t3','t4','t5','t6']
CV_print= ','.join(CV_print)  
lines.append('PRINT FMT=%s STRIDE=10 FILE=COLVAR_flip_out2 ARG=%s \n' %('%8.4f',CV_print))


f = open("ABMD/plumed.dat", "w")
for elem in lines:
    f.writelines(elem)
f.close()

In [58]:
%%bash 
gmx_mpi grompp -f ./ABMD/md_prod.mdp -c ./ABMD/md_0.gro -p ./ABMD/topol.top -n ./ABMD/index.ndx -o ./ABMD/flip_out2.tpr 

             :-) GROMACS - gmx grompp, 2022.5-plumed_2.10.0_dev (-:

Executable:   /home/rizziv@farma.unige.ch/programs/gromacs-2022.5/install_single_cuda/bin/gmx_mpi
Data prefix:  /home/rizziv@farma.unige.ch/programs/gromacs-2022.5/install_single_cuda
Working dir:  /home/frohlkin@farma.unige.ch/Projects/TransitionPathSampling_DeepLNE/Publication_Toymodels/RNA
Command line:
  gmx_mpi grompp -f ./ABMD/md_prod.mdp -c ./ABMD/md_0.gro -p ./ABMD/topol.top -n ./ABMD/index.ndx -o ./ABMD/flip_out2.tpr

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'

NOTE 1 [file ./ABMD/md_prod.mdp]:
  leapfrog does not yet support Nose-Hoover chains, nhchainlength reset to 1

Generating 1-4 interactions: fudge = 0.5
Number of degrees of freedom in T-Coupling group System is 69506.00

There was 1 note

Back Off! I just backed up ./ABMD/flip_out2.tpr to ./ABMD/#flip_out2.tpr.1#

GROMACS reminds you: "Given enough eyeballs, all bugs are shallow." (Linus Torvalds, on the power of open so

Setting the LD random seed to -268548325

Generated 666 of the 666 non-bonded parameter combinations

Generated 666 of the 666 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'system1'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'Na+'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'WAT'

turning H bonds into constraints...

Cleaning up constraints and constant bonded interactions with virtual sites

The largest distance between excluded atoms is 0.410 nm

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.037 nm, buffer size 0.037 nm

Set rlist, assuming 4x4 atom pair-list, to 1.000 nm, buffer size 0.000 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 52x52x52, spacing 0.150 0.150 0.150

Estimate for the relative computational 

In [62]:
%%bash 
cat > ABMD/run.sh << EOF
export OMP_NUM_THREADS=8
mpirun -n 2 gmx_mpi mdrun -deffnm flip_out2 -plumed plumed.dat -pin on -pinoffset 0 -notunepme -nsteps 1000000 -nb gpu &
EOF

In [63]:
!find . -type f -name "#*" -exec rm -rf {} +
!find . -type f -name "step*" -exec rm -rf {} +
!find . -type f -name "bck*" -exec rm -rf {} +

In [73]:
colvar=plumed.read_as_pandas("ABMD/COLVAR_flip_out2")
colvar

+++ Loading the PLUMED kernel runtime +++
+++ PLUMED_KERNEL="/home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumedKernel.so" +++
+++ An error occurred. Message from dlopen(): /home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumedKernel.so: undefined symbol: _ZN5torch3jit6MethodclESt6vectorIN3c106IValueESaIS4_EERKSt13unordered_mapINSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEES4_St4hashISD_ESt8equal_toISD_ESaISt4pairIKSD_S4_EEE +++
+++ This error is expected if you are trying to load a kernel <=2.4
+++ Trying /home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumed.so +++


time  A8G24C25_cmap  A8G7C9_cmap  G7C9_cmap     hldaN      AWO      t2  \
0      0.00         4.1382       7.1411    15.5680  -26.7914   7.6808  1.1280   
1      0.02         4.1947       7.1876    15.5212  -26.8358   7.5616  1.1945   
2      0.04         4.2123       7.2457    15.3827  -26.6762   7.5763  1.2842   
3      0.06         4.2087       7.1874    15.3578  -26.4892   7.5345  1.2483   
4      0.08         4.2242       7.1547    15.2736  -26.5975   7.4747  1.3330   
...     ...            ...          ...        ...       ...      ...     ...   
1907  38.14        19.9404      32.1303     7.0581 -103.9651  15.4160 -2.7581   
1908  38.16        19.9204      32.1078     6.9933 -103.5619  15.2663 -2.7142   
1909  38.18        19.9389      32.0764     7.0349 -103.5028  15.1058 -2.7182   
1910  38.20        19.9933      32.1142     6.9677 -103.6047  15.0111 -2.6823   
1911  38.22        20.0484      32.1652     7.0558 -103.6941  14.9076 -2.6697   

          t3      t4      t5      t6  
0     2.5027 -3.0143 -1.8910 -1.1955  
1     2.7006 -2.8910 -2.1729 -1.1478  
2     2.7179 -2.7963 -2.2825 -1.2511  
3     2.6646 -2.6906 -2.2634 -1.3129  
4     2.7492 -2.8074 -2.3360 -1.2976  
...      ...     ...     ...     ...  
1907  1.6356 -2.7012  1.6699 -1.9031  
1908  1.5327 -2.6303  1.6973 -1.6536  
1909  1.5202 -2.6244  1.6881 -1.7293  
1910  1.5153 -2.6401  1.7388 -1.8482  
1911  1.5681 -2.5798  1.6214 -1.7678  

[1912 rows x 11 columns]

In [67]:
lines=[]

#------------------------------------- additional CVs to track ---------------------------------------
lines.append("""

d1: DISTANCE ATOMS=243,751 #A-G
d2: DISTANCE ATOMS=243,784 #A-C
d3: DISTANCE ATOMS=243,787 #A-C
d4: DISTANCE ATOMS=243,794 #A-RIB
d5: DISTANCE ATOMS=243,800 #A-O
d6: DISTANCE ATOMS=246,751 #A-G
d7: DISTANCE ATOMS=246,784 #A-C
d8: DISTANCE ATOMS=246,787 #A-C
d9: DISTANCE ATOMS=246,794 #A-RIB
d10: DISTANCE ATOMS=246,800 #A-O

d1n: CUSTOM ARG=d1 FUNC=x/2.11 PERIODIC=NO
d2n: CUSTOM ARG=d2 FUNC=x/2.25 PERIODIC=NO
d3n: CUSTOM ARG=d3 FUNC=x/2.17 PERIODIC=NO
d4n: CUSTOM ARG=d4 FUNC=x/2.53 PERIODIC=NO
d5n: CUSTOM ARG=d5 FUNC=x/2.78 PERIODIC=NO
d6n: CUSTOM ARG=d6 FUNC=x/2.07 PERIODIC=NO
d7n: CUSTOM ARG=d7 FUNC=x/2.22 PERIODIC=NO
d8n: CUSTOM ARG=d8 FUNC=x/2.13 PERIODIC=NO
d9n: CUSTOM ARG=d1 FUNC=x/2.49 PERIODIC=NO
d10n: CUSTOM ARG=d10 FUNC=x/2.71 PERIODIC=NO


#hlda: COMBINE ARG=d1,d2,d3,d4,d5,d6,d7,d8,d9,d10 COEFFICIENTS=-14.815268,-15.319241,-13.384423,-7.26893,-7.971926,-16.557427,-14.808337,-14.668623,-9.334746,-9.811748 PERIODIC=NO
hldaN: COMBINE ARG=d1n,d2n,d3n,d4n,d5n,d6n,d7n,d8n,d9n,d10n COEFFICIENTS=-11.63344,-11.311855,-12.316635,-10.216072,-10.565613,-10.745473,-9.307333,-11.834765,-11.317639,-12.201419 PERIODIC=NO

WO: GROUP ATOMS=1020-45543:4 # water molecules
AWO: COORDINATION GROUPA=243 GROUPB=WO SWITCH={RATIONAL D_0=0.0 R_0=0.35 NN=6 MM=10} NLIST NL_CUTOFF=2.0 NL_STRIDE=20

t1: TORSION ATOMS=225,228,229,232 #A, P-O-C-C
t2: TORSION ATOMS=228,229,232,251 #A, O-C-C-C
t3: TORSION ATOMS=229,232,251,257 #A C-C-C_O
t4: TORSION ATOMS=232,251,257,258 #A C-C-O-P
t5: TORSION ATOMS=251,257,258,261 #A C-O-P-O
t6: TORSION ATOMS=257,258,261,262 #A-C  O-P-O-C
t7: TORSION ATOMS=258,261,262,265 #C P-O-C-C
t8: TORSION ATOMS=261,262,265,282 #C O-C-C-C
t9: TORSION ATOMS=262,265,282,288 #C C-C-C-O
t10: TORSION ATOMS=265,282,288,289 #C C-C-O-P

t-1: TORSION ATOMS=218,224,225,228 #G-A C-O-P-O
t-2: TORSION ATOMS=198,218,224,225 #G-A C-C-O-P
    
""")
             
#----------------------------------------------------------------------------------------------


features=[]

temp=[]
for e,elem in enumerate(A8G24C25_cmap):
    lines.append('dA8G24C25_%s: DISTANCE ATOMS=%s,%s \n'%(e+1,elem[0],elem[1]))
    temp.append('dA8G24C25_%s' %(e+1))

temp = ','.join(temp)              
lines.append('A8G24C25_cmap: COMBINE ARG=%s PERIODIC=NO \n' %temp)
features.append('A8G24C25_cmap')

temp=[]
for e,elem in enumerate(A8G7C9_cmap):
    lines.append('dA8G7C9_%s: DISTANCE ATOMS=%s,%s \n'%(e+1,elem[0],elem[1]))
    temp.append('dA8G7C9_%s' %(e+1))

temp = ','.join(temp)              
lines.append('A8G7C9_cmap: COMBINE ARG=%s PERIODIC=NO \n' %temp)
features.append('A8G7C9_cmap')

temp=[]
for e,elem in enumerate(G7C9_cmap):
    lines.append('dG7C9_%s: DISTANCE ATOMS=%s,%s \n'%(e+1,elem[0],elem[1]))
    temp.append('dG7C9_%s' %(e+1))

temp = ','.join(temp)              
lines.append('G7C9_cmap: COMBINE ARG=%s PERIODIC=NO \n' %temp)
features.append('G7C9_cmap')

lines.append('abmd: ABMD ARG=A8G24C25_cmap,A8G7C9_cmap,G7C9_cmap TO=100,100,20 KAPPA=0.001,0.001,100 \n')

lines.append('COMMITTOR ...\n')
lines.append('   ARG=A8G24C25_cmap,A8G7C9_cmap,G7C9_cmap \n' )
lines.append('   STRIDE=10\n')
lines.append('   BASIN_LL1=20,32,15\n')
lines.append('   BASIN_UL1=50,50,20\n')
lines.append('...\n')

CV_print=['A8G24C25_cmap','A8G7C9_cmap','G7C9_cmap','hldaN','AWO','t2','t3','t4','t5','t6']
CV_print= ','.join(CV_print)  
lines.append('PRINT FMT=%s STRIDE=10 FILE=COLVAR_flip_out3 ARG=%s \n' %('%8.4f',CV_print))


f = open("ABMD/plumed.dat", "w")
for elem in lines:
    f.writelines(elem)
f.close()

In [68]:
%%bash 
gmx_mpi grompp -f ./ABMD/md_prod.mdp -c ./ABMD/md_0.gro -p ./ABMD/topol.top -n ./ABMD/index.ndx -o ./ABMD/flip_out3.tpr 

             :-) GROMACS - gmx grompp, 2022.5-plumed_2.10.0_dev (-:

Executable:   /home/rizziv@farma.unige.ch/programs/gromacs-2022.5/install_single_cuda/bin/gmx_mpi
Data prefix:  /home/rizziv@farma.unige.ch/programs/gromacs-2022.5/install_single_cuda
Working dir:  /home/frohlkin@farma.unige.ch/Projects/TransitionPathSampling_DeepLNE/Publication_Toymodels/RNA
Command line:
  gmx_mpi grompp -f ./ABMD/md_prod.mdp -c ./ABMD/md_0.gro -p ./ABMD/topol.top -n ./ABMD/index.ndx -o ./ABMD/flip_out3.tpr

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'

NOTE 1 [file ./ABMD/md_prod.mdp]:
  leapfrog does not yet support Nose-Hoover chains, nhchainlength reset to 1

Generating 1-4 interactions: fudge = 0.5
Number of degrees of freedom in T-Coupling group System is 69506.00

There was 1 note

Back Off! I just backed up ./ABMD/flip_out3.tpr to ./ABMD/#flip_out3.tpr.1#

GROMACS reminds you: "My Head Goes Pop Pop Pop Pop Pop" (F. Black)



Setting the LD random seed to -1100234891

Generated 666 of the 666 non-bonded parameter combinations

Generated 666 of the 666 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'system1'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'Na+'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'WAT'

turning H bonds into constraints...

Cleaning up constraints and constant bonded interactions with virtual sites

The largest distance between excluded atoms is 0.410 nm

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.037 nm, buffer size 0.037 nm

Set rlist, assuming 4x4 atom pair-list, to 1.000 nm, buffer size 0.000 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 52x52x52, spacing 0.150 0.150 0.150

Estimate for the relative computational

In [69]:
%%bash 
cat > ABMD/run.sh << EOF
export OMP_NUM_THREADS=8
mpirun -n 2 gmx_mpi mdrun -deffnm flip_out3 -plumed plumed.dat -pin on -pinoffset 0 -notunepme -nsteps 1000000 -nb gpu &
EOF

In [74]:
!find . -type f -name "#*" -exec rm -rf {} +
!find . -type f -name "step*" -exec rm -rf {} +
!find . -type f -name "bck*" -exec rm -rf {} +

In [85]:
colvar=plumed.read_as_pandas("ABMD/COLVAR_flip_out3")
colvar

+++ Loading the PLUMED kernel runtime +++
+++ PLUMED_KERNEL="/home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumedKernel.so" +++
+++ An error occurred. Message from dlopen(): /home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumedKernel.so: undefined symbol: _ZN5torch3jit6MethodclESt6vectorIN3c106IValueESaIS4_EERKSt13unordered_mapINSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEES4_St4hashISD_ESt8equal_toISD_ESaISt4pairIKSD_S4_EEE +++
+++ This error is expected if you are trying to load a kernel <=2.4
+++ Trying /home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumed.so +++


time  A8G24C25_cmap  A8G7C9_cmap  G7C9_cmap     hldaN      AWO      t2  \
0      0.00         4.1382       7.1411    15.5680  -26.7914   7.6808  1.1280   
1      0.02         4.1956       7.2001    15.6763  -26.8401   7.5582  1.1946   
2      0.04         4.2114       7.3390    15.8941  -26.6680   7.5801  1.2850   
3      0.06         4.2085       7.4287    16.3400  -26.4846   7.5381  1.2511   
4      0.08         4.2276       7.5713    16.8092  -26.5900   7.4768  1.3376   
...     ...            ...          ...        ...       ...      ...     ...   
2409  48.18        19.7958      32.8224    19.7086 -101.9524  13.9722  1.1949   
2410  48.20        19.7522      32.7920    19.6957 -101.4797  13.8689  1.5838   
2411  48.22        19.8283      32.7723    19.8061 -101.5886  13.7934  1.5955   
2412  48.24        19.9323      32.9073    19.7359 -101.9254  13.9393  1.3897   
2413  48.26        20.0142      33.0270    19.8961 -102.2557  14.0352  1.5391   

          t3      t4      t5      t6  
0     2.5027 -3.0143 -1.8910 -1.1955  
1     2.7003 -2.8910 -2.1728 -1.1480  
2     2.7184 -2.7966 -2.2820 -1.2579  
3     2.6630 -2.6912 -2.2620 -1.3264  
4     2.7429 -2.8092 -2.3262 -1.3129  
...      ...     ...     ...     ...  
2409  1.3917  2.9362 -2.9843 -1.6749  
2410  1.2310  2.9687 -3.0499 -1.8015  
2411  1.2025  3.0789  3.1171 -1.8563  
2412  1.4691  3.1169  2.8872 -1.9388  
2413  1.4571 -3.0694  2.8447 -2.0031  

[2414 rows x 11 columns]

In [86]:
from skmatter.feature_selection import FPS

transitionAB1=plumed.read_as_pandas("./ABMD/COLVAR_flip_out1")
transitionAB2=plumed.read_as_pandas("./ABMD/COLVAR_flip_out2")
transitionAB3=plumed.read_as_pandas("./ABMD/COLVAR_flip_out3")

training_batches=np.vstack((transitionAB1.iloc[:,1:4],
                            transitionAB2.iloc[:,1:4],
                            transitionAB3.iloc[:,1:4])
                          )
   
print(training_batches.shape)
selector = FPS(n_to_select=1000,initialize=0)

selector.fit(training_batches[:,:3].T)
r_ndx=selector.selected_idx_

training_datapoints =training_batches[r_ndx,:]
training_datapoints.shape

(7615, 3)


+++ Loading the PLUMED kernel runtime +++
+++ PLUMED_KERNEL="/home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumedKernel.so" +++
+++ An error occurred. Message from dlopen(): /home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumedKernel.so: undefined symbol: _ZN5torch3jit6MethodclESt6vectorIN3c106IValueESaIS4_EERKSt13unordered_mapINSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEES4_St4hashISD_ESt8equal_toISD_ESaISt4pairIKSD_S4_EEE +++
+++ This error is expected if you are trying to load a kernel <=2.4
+++ Trying /home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumed.so +++
+++ Loading the PLUMED kernel runtime +++
+++ PLUMED_KERNEL="/home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumedKernel.so" +++
+++ An error occurred. Message from dlopen(): /home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumedKernel.so: undefined symbol: _ZN5torch3jit6MethodclESt6vectorIN3c106IValueESaIS4_EERKSt13unordered_mapINSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEES4_St4hashISD_ESt

(1000, 3)

In [87]:
import plotly.graph_objects as go

phi1=training_datapoints[:,0]
phi2=training_datapoints[:,1]
phi3=training_datapoints[:,2]


fig = go.Figure(data=go.Scatter3d(
    x=phi1[:],
    y=phi2[:],
    z=phi3[:],
    mode='markers',
    marker=dict(
        size=5,
        color='black',  
        opacity=0.5)
    )
)

fig.update_layout(
    scene=dict(
        xaxis=dict(backgroundcolor='#CCCCCC'),
        yaxis=dict(backgroundcolor='#CCCCCC'),
        zaxis=dict(backgroundcolor='#CCCCCC', gridcolor='rgba(0, 0, 0, 0)')
    )
)

fig.update_layout(
    scene=dict(
        xaxis=dict(title=r'CV1'),
        yaxis=dict(title=r'CV2'),
        zaxis=dict(title=r'CV3')
    )
)

fig.show()


In [88]:
#get all the starting structures for flipping back in

In [89]:
%%bash
gmx_mpi check -f ABMD/flip_out1.xtc

             :-) GROMACS - gmx check, 2022.5-plumed_2.10.0_dev (-:

Executable:   /home/rizziv@farma.unige.ch/programs/gromacs-2022.5/install_single_cuda/bin/gmx_mpi
Data prefix:  /home/rizziv@farma.unige.ch/programs/gromacs-2022.5/install_single_cuda
Working dir:  /home/frohlkin@farma.unige.ch/Projects/TransitionPathSampling_DeepLNE/Publication_Toymodels/RNA
Command line:
  gmx_mpi check -f ABMD/flip_out1.xtc

Reading frame       0 time    0.000   
# Atoms  45543
Precision 0.001 (nm)
Reading frame     300 time   60.000   


Item        #frames Timestep (ps)
Step           329    0.2
Time           329    0.2
Lambda           0
Coords         329    0.2
Velocities       0
Forces           0
Box            329    0.2

GROMACS reminds you: "I Calculate My Birthright" (P.J. Harvey)



Checking file ABMD/flip_out1.xtc


In [90]:
print(329*0.2)

65.8


In [91]:
%%bash
echo 0| gmx_mpi trjconv -f ABMD/flip_out1.xtc -s ABMD/md_0.gro -dump 65.8 -o ABMD/flip_out1.pdb

            :-) GROMACS - gmx trjconv, 2022.5-plumed_2.10.0_dev (-:

Executable:   /home/rizziv@farma.unige.ch/programs/gromacs-2022.5/install_single_cuda/bin/gmx_mpi
Data prefix:  /home/rizziv@farma.unige.ch/programs/gromacs-2022.5/install_single_cuda
Working dir:  /home/frohlkin@farma.unige.ch/Projects/TransitionPathSampling_DeepLNE/Publication_Toymodels/RNA
Command line:
  gmx_mpi trjconv -f ABMD/flip_out1.xtc -s ABMD/md_0.gro -dump 65.8 -o ABMD/flip_out1.pdb

Will write pdb: Protein data bank file
Group     0 (         System) has 45543 elements
Group     1 (          Other) has    94 elements
Group     2 (             G5) has    32 elements
Group     3 (             C3) has    32 elements
Group     4 (            Na+) has    30 elements
Group     5 (          Water) has 44524 elements
Group     6 (            SOL) has 44524 elements
Group     7 (      non-Water) has  1019 elements
Reading frame       0 time    0.000   
Precision of ABMD/flip_out1.xtc is 0.001 (nm)
Reading frame   

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 0: 'System'


In [92]:
%%bash
gmx_mpi check -f ABMD/flip_out2.xtc

             :-) GROMACS - gmx check, 2022.5-plumed_2.10.0_dev (-:

Executable:   /home/rizziv@farma.unige.ch/programs/gromacs-2022.5/install_single_cuda/bin/gmx_mpi
Data prefix:  /home/rizziv@farma.unige.ch/programs/gromacs-2022.5/install_single_cuda
Working dir:  /home/frohlkin@farma.unige.ch/Projects/TransitionPathSampling_DeepLNE/Publication_Toymodels/RNA
Command line:
  gmx_mpi check -f ABMD/flip_out2.xtc

Reading frame       0 time    0.000   
# Atoms  45543
Precision 0.001 (nm)
Reading frame     190 time   38.000   


Item        #frames Timestep (ps)
Step           192    0.2
Time           192    0.2
Lambda           0
Coords         192    0.2
Velocities       0
Forces           0
Box            192    0.2

GROMACS reminds you: "And You Will Know That My Name is the Lord When I Lay My Vengeance Upon Thee." (Pulp Fiction)



Checking file ABMD/flip_out2.xtc


In [93]:
print(192*0.2)

38.400000000000006


In [94]:
%%bash
echo 0| gmx_mpi trjconv -f ABMD/flip_out2.xtc -s ABMD/md_0.gro -dump 38.4 -o ABMD/flip_out2.pdb

            :-) GROMACS - gmx trjconv, 2022.5-plumed_2.10.0_dev (-:

Executable:   /home/rizziv@farma.unige.ch/programs/gromacs-2022.5/install_single_cuda/bin/gmx_mpi
Data prefix:  /home/rizziv@farma.unige.ch/programs/gromacs-2022.5/install_single_cuda
Working dir:  /home/frohlkin@farma.unige.ch/Projects/TransitionPathSampling_DeepLNE/Publication_Toymodels/RNA
Command line:
  gmx_mpi trjconv -f ABMD/flip_out2.xtc -s ABMD/md_0.gro -dump 38.4 -o ABMD/flip_out2.pdb

Will write pdb: Protein data bank file
Group     0 (         System) has 45543 elements
Group     1 (          Other) has    94 elements
Group     2 (             G5) has    32 elements
Group     3 (             C3) has    32 elements
Group     4 (            Na+) has    30 elements
Group     5 (          Water) has 44524 elements
Group     6 (            SOL) has 44524 elements
Group     7 (      non-Water) has  1019 elements
Reading frame       0 time    0.000   
Precision of ABMD/flip_out2.xtc is 0.001 (nm)
Reading frame   

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 0: 'System'


In [95]:
%%bash
gmx_mpi check -f ABMD/flip_out3.xtc

             :-) GROMACS - gmx check, 2022.5-plumed_2.10.0_dev (-:

Executable:   /home/rizziv@farma.unige.ch/programs/gromacs-2022.5/install_single_cuda/bin/gmx_mpi
Data prefix:  /home/rizziv@farma.unige.ch/programs/gromacs-2022.5/install_single_cuda
Working dir:  /home/frohlkin@farma.unige.ch/Projects/TransitionPathSampling_DeepLNE/Publication_Toymodels/RNA
Command line:
  gmx_mpi check -f ABMD/flip_out3.xtc

Reading frame       0 time    0.000   
# Atoms  45543
Precision 0.001 (nm)
Reading frame     200 time   40.000   


Item        #frames Timestep (ps)
Step           242    0.2
Time           242    0.2
Lambda           0
Coords         242    0.2
Velocities       0
Forces           0
Box            242    0.2

GROMACS reminds you: "Screw a Lightbulb in your Head" (Gogol Bordello)



Checking file ABMD/flip_out3.xtc


In [96]:
print(242*0.2)

48.400000000000006


In [97]:
%%bash
echo 0| gmx_mpi trjconv -f ABMD/flip_out3.xtc -s ABMD/md_0.gro -dump 48.4 -o ABMD/flip_out3.pdb

            :-) GROMACS - gmx trjconv, 2022.5-plumed_2.10.0_dev (-:

Executable:   /home/rizziv@farma.unige.ch/programs/gromacs-2022.5/install_single_cuda/bin/gmx_mpi
Data prefix:  /home/rizziv@farma.unige.ch/programs/gromacs-2022.5/install_single_cuda
Working dir:  /home/frohlkin@farma.unige.ch/Projects/TransitionPathSampling_DeepLNE/Publication_Toymodels/RNA
Command line:
  gmx_mpi trjconv -f ABMD/flip_out3.xtc -s ABMD/md_0.gro -dump 48.4 -o ABMD/flip_out3.pdb

Will write pdb: Protein data bank file
Group     0 (         System) has 45543 elements
Group     1 (          Other) has    94 elements
Group     2 (             G5) has    32 elements
Group     3 (             C3) has    32 elements
Group     4 (            Na+) has    30 elements
Group     5 (          Water) has 44524 elements
Group     6 (            SOL) has 44524 elements
Group     7 (      non-Water) has  1019 elements
Reading frame       0 time    0.000   
Precision of ABMD/flip_out3.xtc is 0.001 (nm)
Reading frame   

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 0: 'System'


In [98]:
%%bash 
gmx_mpi grompp -f ./ABMD/md_prod.mdp -c ./ABMD/flip_out1.pdb -p ./ABMD/topol.top -n ./ABMD/index.ndx -o ./ABMD/flip_in1.tpr 

             :-) GROMACS - gmx grompp, 2022.5-plumed_2.10.0_dev (-:

Executable:   /home/rizziv@farma.unige.ch/programs/gromacs-2022.5/install_single_cuda/bin/gmx_mpi
Data prefix:  /home/rizziv@farma.unige.ch/programs/gromacs-2022.5/install_single_cuda
Working dir:  /home/frohlkin@farma.unige.ch/Projects/TransitionPathSampling_DeepLNE/Publication_Toymodels/RNA
Command line:
  gmx_mpi grompp -f ./ABMD/md_prod.mdp -c ./ABMD/flip_out1.pdb -p ./ABMD/topol.top -n ./ABMD/index.ndx -o ./ABMD/flip_in1.tpr

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'

NOTE 1 [file ./ABMD/md_prod.mdp]:
  leapfrog does not yet support Nose-Hoover chains, nhchainlength reset to 1

Generating 1-4 interactions: fudge = 0.5
Number of degrees of freedom in T-Coupling group System is 69506.00

There was 1 note

GROMACS reminds you: "I like to wait, then I feel like I do something" (Carl Caleman)



Setting the LD random seed to 1568602859

Generated 666 of the 666 non-bonded parameter combinations

Generated 666 of the 666 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'system1'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'Na+'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'WAT'

turning H bonds into constraints...

Cleaning up constraints and constant bonded interactions with virtual sites

The largest distance between excluded atoms is 0.413 nm

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.037 nm, buffer size 0.037 nm

Set rlist, assuming 4x4 atom pair-list, to 1.000 nm, buffer size 0.000 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 52x52x52, spacing 0.150 0.150 0.150

Estimate for the relative computational 

In [99]:
%%bash 
gmx_mpi grompp -f ./ABMD/md_prod.mdp -c ./ABMD/flip_out2.pdb -p ./ABMD/topol.top -n ./ABMD/index.ndx -o ./ABMD/flip_in2.tpr 

             :-) GROMACS - gmx grompp, 2022.5-plumed_2.10.0_dev (-:

Executable:   /home/rizziv@farma.unige.ch/programs/gromacs-2022.5/install_single_cuda/bin/gmx_mpi
Data prefix:  /home/rizziv@farma.unige.ch/programs/gromacs-2022.5/install_single_cuda
Working dir:  /home/frohlkin@farma.unige.ch/Projects/TransitionPathSampling_DeepLNE/Publication_Toymodels/RNA
Command line:
  gmx_mpi grompp -f ./ABMD/md_prod.mdp -c ./ABMD/flip_out2.pdb -p ./ABMD/topol.top -n ./ABMD/index.ndx -o ./ABMD/flip_in2.tpr

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'

NOTE 1 [file ./ABMD/md_prod.mdp]:
  leapfrog does not yet support Nose-Hoover chains, nhchainlength reset to 1

Generating 1-4 interactions: fudge = 0.5
Number of degrees of freedom in T-Coupling group System is 69506.00

There was 1 note

GROMACS reminds you: "Do you have mole problems? If so, call Avogadro at 602-1023." (Jay Leno)



Setting the LD random seed to 1463015925

Generated 666 of the 666 non-bonded parameter combinations

Generated 666 of the 666 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'system1'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'Na+'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'WAT'

turning H bonds into constraints...

Cleaning up constraints and constant bonded interactions with virtual sites

The largest distance between excluded atoms is 0.408 nm

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.037 nm, buffer size 0.037 nm

Set rlist, assuming 4x4 atom pair-list, to 1.000 nm, buffer size 0.000 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 52x52x52, spacing 0.149 0.149 0.149

Estimate for the relative computational 

In [100]:
%%bash 
gmx_mpi grompp -f ./ABMD/md_prod.mdp -c ./ABMD/flip_out3.pdb -p ./ABMD/topol.top -n ./ABMD/index.ndx -o ./ABMD/flip_in3.tpr 

             :-) GROMACS - gmx grompp, 2022.5-plumed_2.10.0_dev (-:

Executable:   /home/rizziv@farma.unige.ch/programs/gromacs-2022.5/install_single_cuda/bin/gmx_mpi
Data prefix:  /home/rizziv@farma.unige.ch/programs/gromacs-2022.5/install_single_cuda
Working dir:  /home/frohlkin@farma.unige.ch/Projects/TransitionPathSampling_DeepLNE/Publication_Toymodels/RNA
Command line:
  gmx_mpi grompp -f ./ABMD/md_prod.mdp -c ./ABMD/flip_out3.pdb -p ./ABMD/topol.top -n ./ABMD/index.ndx -o ./ABMD/flip_in3.tpr

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'

NOTE 1 [file ./ABMD/md_prod.mdp]:
  leapfrog does not yet support Nose-Hoover chains, nhchainlength reset to 1

Generating 1-4 interactions: fudge = 0.5
Number of degrees of freedom in T-Coupling group System is 69506.00

There was 1 note

GROMACS reminds you: "There is only one thing worse than coming home from the lab to a sink full of dirty dishes, and that is not going to the lab at all!" (Chien-Shiung Wu)



Setting the LD random seed to -745545761

Generated 666 of the 666 non-bonded parameter combinations

Generated 666 of the 666 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'system1'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'Na+'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'WAT'

turning H bonds into constraints...

Cleaning up constraints and constant bonded interactions with virtual sites

The largest distance between excluded atoms is 0.409 nm

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.037 nm, buffer size 0.037 nm

Set rlist, assuming 4x4 atom pair-list, to 1.000 nm, buffer size 0.000 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 52x52x52, spacing 0.150 0.150 0.150

Estimate for the relative computational 

In [101]:
colvar=plumed.read_as_pandas("COLVAR_stateA")

print(colvar['A8G24C25_cmap'].to_numpy())
print(colvar['A8G7C9_cmap'].to_numpy())
print(colvar['G7C9_cmap'].to_numpy())
print(colvar['hldaN'].to_numpy())
print(colvar['AWO'].to_numpy())

[4.1382]
[7.1411]
[15.568]
[-26.7914]
[7.6808]


+++ Loading the PLUMED kernel runtime +++
+++ PLUMED_KERNEL="/home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumedKernel.so" +++
+++ An error occurred. Message from dlopen(): /home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumedKernel.so: undefined symbol: _ZN5torch3jit6MethodclESt6vectorIN3c106IValueESaIS4_EERKSt13unordered_mapINSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEES4_St4hashISD_ESt8equal_toISD_ESaISt4pairIKSD_S4_EEE +++
+++ This error is expected if you are trying to load a kernel <=2.4
+++ Trying /home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumed.so +++


In [111]:
lines=[]

#------------------------------------- additional CVs to track ---------------------------------------
lines.append("""

d1: DISTANCE ATOMS=243,751 #A-G
d2: DISTANCE ATOMS=243,784 #A-C
d3: DISTANCE ATOMS=243,787 #A-C
d4: DISTANCE ATOMS=243,794 #A-RIB
d5: DISTANCE ATOMS=243,800 #A-O
d6: DISTANCE ATOMS=246,751 #A-G
d7: DISTANCE ATOMS=246,784 #A-C
d8: DISTANCE ATOMS=246,787 #A-C
d9: DISTANCE ATOMS=246,794 #A-RIB
d10: DISTANCE ATOMS=246,800 #A-O

d1n: CUSTOM ARG=d1 FUNC=x/2.11 PERIODIC=NO
d2n: CUSTOM ARG=d2 FUNC=x/2.25 PERIODIC=NO
d3n: CUSTOM ARG=d3 FUNC=x/2.17 PERIODIC=NO
d4n: CUSTOM ARG=d4 FUNC=x/2.53 PERIODIC=NO
d5n: CUSTOM ARG=d5 FUNC=x/2.78 PERIODIC=NO
d6n: CUSTOM ARG=d6 FUNC=x/2.07 PERIODIC=NO
d7n: CUSTOM ARG=d7 FUNC=x/2.22 PERIODIC=NO
d8n: CUSTOM ARG=d8 FUNC=x/2.13 PERIODIC=NO
d9n: CUSTOM ARG=d1 FUNC=x/2.49 PERIODIC=NO
d10n: CUSTOM ARG=d10 FUNC=x/2.71 PERIODIC=NO


#hlda: COMBINE ARG=d1,d2,d3,d4,d5,d6,d7,d8,d9,d10 COEFFICIENTS=-14.815268,-15.319241,-13.384423,-7.26893,-7.971926,-16.557427,-14.808337,-14.668623,-9.334746,-9.811748 PERIODIC=NO
hldaN: COMBINE ARG=d1n,d2n,d3n,d4n,d5n,d6n,d7n,d8n,d9n,d10n COEFFICIENTS=-11.63344,-11.311855,-12.316635,-10.216072,-10.565613,-10.745473,-9.307333,-11.834765,-11.317639,-12.201419 PERIODIC=NO

WO: GROUP ATOMS=1020-45543:4 # water molecules
AWO: COORDINATION GROUPA=243 GROUPB=WO SWITCH={RATIONAL D_0=0.0 R_0=0.35 NN=6 MM=10} NLIST NL_CUTOFF=2.0 NL_STRIDE=20

t1: TORSION ATOMS=225,228,229,232 #A, P-O-C-C
t2: TORSION ATOMS=228,229,232,251 #A, O-C-C-C
t3: TORSION ATOMS=229,232,251,257 #A C-C-C_O
t4: TORSION ATOMS=232,251,257,258 #A C-C-O-P
t5: TORSION ATOMS=251,257,258,261 #A C-O-P-O
t6: TORSION ATOMS=257,258,261,262 #A-C  O-P-O-C
t7: TORSION ATOMS=258,261,262,265 #C P-O-C-C
t8: TORSION ATOMS=261,262,265,282 #C O-C-C-C
t9: TORSION ATOMS=262,265,282,288 #C C-C-C-O
t10: TORSION ATOMS=265,282,288,289 #C C-C-O-P

t-1: TORSION ATOMS=218,224,225,228 #G-A C-O-P-O
t-2: TORSION ATOMS=198,218,224,225 #G-A C-C-O-P
    
""")
             
#----------------------------------------------------------------------------------------------


features=[]

temp=[]
for e,elem in enumerate(A8G24C25_cmap):
    lines.append('dA8G24C25_%s: DISTANCE ATOMS=%s,%s \n'%(e+1,elem[0],elem[1]))
    temp.append('dA8G24C25_%s' %(e+1))

temp = ','.join(temp)              
lines.append('A8G24C25_cmap: COMBINE ARG=%s PERIODIC=NO \n' %temp)
features.append('A8G24C25_cmap')

temp=[]
for e,elem in enumerate(A8G7C9_cmap):
    lines.append('dA8G7C9_%s: DISTANCE ATOMS=%s,%s \n'%(e+1,elem[0],elem[1]))
    temp.append('dA8G7C9_%s' %(e+1))

temp = ','.join(temp)              
lines.append('A8G7C9_cmap: COMBINE ARG=%s PERIODIC=NO \n' %temp)
features.append('A8G7C9_cmap')

temp=[]
for e,elem in enumerate(G7C9_cmap):
    lines.append('dG7C9_%s: DISTANCE ATOMS=%s,%s \n'%(e+1,elem[0],elem[1]))
    temp.append('dG7C9_%s' %(e+1))

temp = ','.join(temp)              
lines.append('G7C9_cmap: COMBINE ARG=%s PERIODIC=NO \n' %temp)
features.append('G7C9_cmap')



lines.append('abmd: ABMD ARG=A8G24C25_cmap,A8G7C9_cmap,G7C9_cmap  TO=0,0,15.568 KAPPA=1,1,5 \n')

lines.append('COMMITTOR ...\n')
lines.append('   ARG=A8G24C25_cmap,A8G7C9_cmap \n' )
lines.append('   STRIDE=100\n')
lines.append('   BASIN_LL1=0,0\n')
lines.append('   BASIN_UL1=4.1382,7.1411\n')
lines.append('...\n')

CV_print=['A8G24C25_cmap','A8G7C9_cmap','G7C9_cmap','hldaN','AWO','t2','t3','t4','t5','t6']
CV_print= ','.join(CV_print)  
lines.append('PRINT FMT=%s STRIDE=10 FILE=COLVAR_flip_in3 ARG=%s \n' %('%8.4f',CV_print))


f = open("ABMD/plumed.dat", "w")
for elem in lines:
    f.writelines(elem)
f.close()

In [112]:
%%bash 
cat > ABMD/run.sh << EOF
export OMP_NUM_THREADS=8
mpirun -n 2 gmx_mpi mdrun -deffnm flip_in3 -plumed plumed.dat -pin on -pinoffset 0 -notunepme -nsteps 1000000 -nb gpu &
EOF

In [ ]:
#mpirun -n 2 gmx_mpi mdrun -deffnm flip_in1 -plumed plumed.dat -pin on -pinoffset 0 -notunepme -nsteps 1000000 -nb gpu &
#mpirun -n 2 gmx_mpi mdrun -deffnm flip_in2 -plumed plumed.dat -pin on -pinoffset 0 -notunepme -nsteps 1000000 -nb gpu &
#mpirun -n 2 gmx_mpi mdrun -deffnm flip_in3 -plumed plumed.dat -pin on -pinoffset 0 -notunepme -nsteps 1000000 -nb gpu &

In [113]:
!find . -type f -name "#*" -exec rm -rf {} +
!find . -type f -name "step*" -exec rm -rf {} +
!find . -type f -name "bck*" -exec rm -rf {} +

In [116]:
from skmatter.feature_selection import FPS

transitionAB1=plumed.read_as_pandas("./ABMD/COLVAR_flip_out1")
transitionAB2=plumed.read_as_pandas("./ABMD/COLVAR_flip_out2")
transitionAB3=plumed.read_as_pandas("./ABMD/COLVAR_flip_out3")

transitionBA1=plumed.read_as_pandas("./ABMD/COLVAR_flip_in1")
transitionBA2=plumed.read_as_pandas("./ABMD/COLVAR_flip_in2")
transitionBA3=plumed.read_as_pandas("./ABMD/COLVAR_flip_in3")

training_batches=np.vstack((transitionBA1.iloc[:,1:4],
                            transitionBA2.iloc[:,1:4],
                            transitionBA3.iloc[:,1:4])
                          )
   
print(training_batches.shape)
selector = FPS(n_to_select=1000,initialize=0)

selector.fit(training_batches[:,:3].T)
r_ndx=selector.selected_idx_

training_datapoints =training_batches[r_ndx,:]
training_datapoints.shape

(2263, 3)


+++ Loading the PLUMED kernel runtime +++
+++ PLUMED_KERNEL="/home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumedKernel.so" +++
+++ An error occurred. Message from dlopen(): /home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumedKernel.so: undefined symbol: _ZN5torch3jit6MethodclESt6vectorIN3c106IValueESaIS4_EERKSt13unordered_mapINSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEES4_St4hashISD_ESt8equal_toISD_ESaISt4pairIKSD_S4_EEE +++
+++ This error is expected if you are trying to load a kernel <=2.4
+++ Trying /home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumed.so +++
+++ Loading the PLUMED kernel runtime +++
+++ PLUMED_KERNEL="/home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumedKernel.so" +++
+++ An error occurred. Message from dlopen(): /home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumedKernel.so: undefined symbol: _ZN5torch3jit6MethodclESt6vectorIN3c106IValueESaIS4_EERKSt13unordered_mapINSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEES4_St4hashISD_ESt

(1000, 3)

In [117]:
import plotly.graph_objects as go

phi1=training_datapoints[:,0]
phi2=training_datapoints[:,1]
phi3=training_datapoints[:,2]


fig = go.Figure(data=go.Scatter3d(
    x=phi1[:],
    y=phi2[:],
    z=phi3[:],
    mode='markers',
    marker=dict(
        size=5,
        color='black',  
        opacity=0.5)
    )
)

fig.update_layout(
    scene=dict(
        xaxis=dict(backgroundcolor='#CCCCCC'),
        yaxis=dict(backgroundcolor='#CCCCCC'),
        zaxis=dict(backgroundcolor='#CCCCCC', gridcolor='rgba(0, 0, 0, 0)')
    )
)

fig.update_layout(
    scene=dict(
        xaxis=dict(title=r'CV1'),
        yaxis=dict(title=r'CV2'),
        zaxis=dict(title=r'CV3')
    )
)

fig.show()


In [118]:
from skmatter.feature_selection import FPS

transitionAB1=plumed.read_as_pandas("./ABMD/COLVAR_flip_out1")
transitionAB2=plumed.read_as_pandas("./ABMD/COLVAR_flip_out2")
transitionAB3=plumed.read_as_pandas("./ABMD/COLVAR_flip_out3")

transitionBA1=plumed.read_as_pandas("./ABMD/COLVAR_flip_in1")
transitionBA2=plumed.read_as_pandas("./ABMD/COLVAR_flip_in2")
transitionBA3=plumed.read_as_pandas("./ABMD/COLVAR_flip_in3")

training_batches=np.vstack((transitionAB1.iloc[:,1:4],
                            transitionAB2.iloc[:,1:4],
                            transitionAB3.iloc[:,1:4],
                            transitionBA1.iloc[:,1:4],
                            transitionBA2.iloc[:,1:4],
                            transitionBA3.iloc[:,1:4])
                          )
   
print(training_batches.shape)
selector = FPS(n_to_select=1000,initialize=0)

selector.fit(training_batches[:,:3].T)
r_ndx=selector.selected_idx_

training_datapoints =training_batches[r_ndx,:]
training_datapoints.shape

(9878, 3)


+++ Loading the PLUMED kernel runtime +++
+++ PLUMED_KERNEL="/home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumedKernel.so" +++
+++ An error occurred. Message from dlopen(): /home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumedKernel.so: undefined symbol: _ZN5torch3jit6MethodclESt6vectorIN3c106IValueESaIS4_EERKSt13unordered_mapINSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEES4_St4hashISD_ESt8equal_toISD_ESaISt4pairIKSD_S4_EEE +++
+++ This error is expected if you are trying to load a kernel <=2.4
+++ Trying /home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumed.so +++
+++ Loading the PLUMED kernel runtime +++
+++ PLUMED_KERNEL="/home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumedKernel.so" +++
+++ An error occurred. Message from dlopen(): /home/rizziv@farma.unige.ch/plumed2-2.9/src/lib/libplumedKernel.so: undefined symbol: _ZN5torch3jit6MethodclESt6vectorIN3c106IValueESaIS4_EERKSt13unordered_mapINSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEES4_St4hashISD_ESt

(1000, 3)

In [119]:
import plotly.graph_objects as go

phi1=training_datapoints[:,0]
phi2=training_datapoints[:,1]
phi3=training_datapoints[:,2]


fig = go.Figure(data=go.Scatter3d(
    x=phi1[:],
    y=phi2[:],
    z=phi3[:],
    mode='markers',
    marker=dict(
        size=5,
        color='black',  
        opacity=0.5)
    )
)

fig.update_layout(
    scene=dict(
        xaxis=dict(backgroundcolor='#CCCCCC'),
        yaxis=dict(backgroundcolor='#CCCCCC'),
        zaxis=dict(backgroundcolor='#CCCCCC', gridcolor='rgba(0, 0, 0, 0)')
    )
)

fig.update_layout(
    scene=dict(
        xaxis=dict(title=r'CV1'),
        yaxis=dict(title=r'CV2'),
        zaxis=dict(title=r'CV3')
    )
)

fig.show()
